In [1]:
import pygame
import os
from math import sin, cos, pi, sqrt
from pygame.locals import *
from random import randrange
FPS = pygame.time.Clock()
pygame.init()
# physics init
thrust_power = 0.04
thrust_diff = 0.003
m = 1
length_arm = 25
Frames = 60
Screen_width = 800
Screen_height = 800
g = 0.08
drone_w = 80
balloon_width = 30
balloon_animation_rate = 0.1
drone_animation_rate = 0.3
first_cloud_x = 140
first_cloud_y = 210
first_cloud_speed = 0.9
second_cloud_x = 300
second_cloud_y = 550
second_cloud_speed = -0.3
first_sun_x = 630
first_sun_y = -100
# game parameters
game_counter = 0
passed_time = 0
steps = 0
max_time = 100
is_dead = False
max_timer_respawn = 3
curr_timer_respawn = 3
theta = 0
omega = 0
alpha = 0
x_coordinate = 400
speed_x = 0
acc_x = 0
y_coordinate = 400
speed_y= 0
acc_y=0
display_screen = pygame.display.set_mode((Screen_width, Screen_height))

drone_image = []
for i in range(1, 4):
    img = pygame.image.load(os.path.join("assets/drone-" + str(i) + ".png"))
    img.convert()
    drone_image.append(
    pygame.transform.scale(img, (drone_w, int(drone_w * 0.30))))
    
balloon_image = []
for i in range(1, 8):
    #img = pygame.image.load(os.path.join("assets/sprites/target-old" + str(i) + ".png"))
    img = pygame.image.load(os.path.join("assets/sprites/target_old.png"))
    img.convert()
    balloon_image.append(pygame.transform.scale(img, (balloon_width, int(balloon_width * 1.73))))
    
# Bg
pygame.font.init()
font_information = pygame.font.Font("assets/Roboto-Regular.ttf", 30)
font_respawning = pygame.font.Font("assets/Roboto-Bold.ttf", 90)
balloon_x = randrange(200, 600)
balloon_y = randrange(200, 600)

first_cloud = pygame.image.load(os.path.join("background-elements/cloud-1.png"))
second_cloud = pygame.image.load(os.path.join("background-elements/cloud-2.png"))
first_sun = pygame.image.load(os.path.join("background-elements/sun.png"))

first_cloud.set_alpha(120)
second_cloud.set_alpha(120)
first_sun.set_alpha(124)

while True:
    pygame.event.get()
    display_screen.fill((173, 220, 255))
    passed_time += 1 / 60
    steps += 1
    display_screen.blit(first_sun, (first_sun_x, first_sun_y))
    first_cloud_x += first_cloud_speed
    if first_cloud_x > Screen_width:
        first_cloud_x = -first_cloud.get_width()
    display_screen.blit(first_cloud, (first_cloud_x, first_cloud_y))
    second_cloud_x += second_cloud_speed
    if second_cloud_x < -second_cloud.get_width():
        second_cloud_x = Screen_width
    display_screen.blit(second_cloud, (second_cloud_x, second_cloud_y))
    if is_dead == False:
        acc_x = 0
        acc_y = g
        alpha = 0
        thrust_left = 0.04
        thrust_right = 0.04
        key_pressed = pygame.key.get_pressed()
        if key_pressed[K_UP]:
            thrust_left += thrust_power
            thrust_right += thrust_power
        if key_pressed[K_DOWN]:
            thrust_left -= thrust_power
            thrust_right -= thrust_power
        if key_pressed[K_LEFT]:
            thrust_left -= thrust_diff
        if key_pressed[K_RIGHT]:
            thrust_right -= thrust_diff
        # calc acc(x,y) and alpha to figure out v(x,y), omega and pos(x,y), theta
        acc_x += (
        -(thrust_left + thrust_right) * sin(theta * pi / 180) / m
        )
        acc_y += (
        -(thrust_left + thrust_right) * cos(theta * pi / 180) / m
        )
        alpha += length_arm * (thrust_right - thrust_left) / m
        speed_x += acc_x
        speed_y += acc_y
        omega += alpha
        x_coordinate += speed_x
        y_coordinate += speed_y
        theta += omega
        distance_fromballoon = sqrt((x_coordinate - balloon_x) ** 2 + (y_coordinate - balloon_y) ** 2)
        if distance_fromballoon < 50:
            balloon_x = randrange(200, 600)
            balloon_y = randrange(200, 600)
            game_counter += 1
        elif distance_fromballoon > 1000:
            is_dead = True
            curr_timer_respawn = max_timer_respawn
    else:
        text_for_respawn = font_respawning.render(
            str(int(curr_timer_respawn) + 1), True, (255, 255, 255)
        )
        text_for_respawn.set_alpha(124)
        display_screen.blit(
        text_for_respawn,
            (
                Screen_width / 2 - text_for_respawn.get_width() / 2,
                Screen_height / 2 - text_for_respawn.get_height() / 2,
            ),
        )
        # reduce curr timer and display it
        curr_timer_respawn -= 1 / 60
        if curr_timer_respawn < 0:
            is_dead = False
            # reset the drone parameters
            theta = 0
            omega = 0
            alpha = 0
            x_coordinate = 400
            speed_x = 0
            acc_x = 0
            y_coordinate = 400
            speed_y= 0
            acc_y=0
            # time over
            
    if passed_time > max_time:
        break
        
    # target
    balloon_curr_frame = balloon_image[int(steps * balloon_animation_rate) % len(balloon_image)]
    display_screen.blit(
        balloon_curr_frame,
        (
            balloon_x - int(balloon_curr_frame.get_width() / 2),
            balloon_y - int(balloon_curr_frame.get_height() / 2),
        ),
        )
    # display drone with correct coordinates and angle
    drone_curr_frame = drone_image[int(steps * drone_animation_rate) % len(drone_image)]
    drone_curr_rotated_frame = pygame.transform.rotate(drone_curr_frame, theta)
    display_screen.blit(
        drone_curr_rotated_frame,
        (
            x_coordinate - int(drone_curr_rotated_frame.get_width() / 2),
            y_coordinate - int(drone_curr_rotated_frame.get_height() / 2),
        ),
        )
    # display collected info
    collected_info = font_information.render("Total collected : " + str(game_counter), True, (255, 255, 255))
    display_screen.blit(collected_info, (20, 20))
    
    time_text = font_information.render("Time elapsed : " + str(int(max_time - passed_time)), True, (255, 255, 255))
    display_screen.blit(time_text, (20, 60))
    pygame.display.update()
    FPS.tick(Frames)


pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
